In [7]:
import pandas as pd
import re

In [23]:
df = pd.DataFrame(
    {
        'group' : ['Vantage_Hedge\\MAM\\MAM_', 'Vantage_Hedge\\MAM\\MNT_', 'Vantage_Hedge\\MAM\\SAM_']
    }
)

In [40]:
from functools import partialmethod
pd.DataFrame.query = partialmethod(pd.DataFrame.query, engine="python")
df = pd.DataFrame(
    {
        'group' : ['Vantage_Hedge\\MAM\\MAM_', 'Vantage_Hedge\\MAM\\MNT_', 'Vantage_Hedge\\MAM\\SAM_']
    }
)
mam_rule = r'^.*[\\]+.*[\\]+MAM'
# df.group.str.contains(r'^.*[\\]+.*[\\]+MAM', regex = True)

# df.group.str.contains(r'^.*\\.*\\MAM', regex = True)
df.query("group.str.match('^.*\\.*\\\MAM')")

# df.query(f"group.str.match('^.*[\\]+.*[\\]+MAM')")





error: bad escape \M at position 6

In [13]:
text = r"""
Vantage_Hedge\\MAM\\MAM_
"""
# Vantage_Hedge\MAM\MAM_
# Vantage_Hedge\MAM\SAM_
pattern = re.compile(r'^.*[\\]+.*[\\]+MAM')
re.findall(pattern, string = text)
matches = re.finditer(pattern, text)
matches
for match in matches:
    print(match)

In [100]:
snippets = r"""

.       - Any Character Except New Line
\d      - Digit (0-9)
\D      - Not a Digit (0-9)
\w      - Word Character (a-z, A-Z, 0-9, _)
\W      - Not a Word Character
\s      - Whitespace (space, tab, newline)
\S      - Not Whitespace (space, tab, newline)

\b      - Word Boundary
\B      - Not a Word Boundary
^       - Beginning of a String
$       - End of a String

[]      - Matches Characters in brackets
[^ ]    - Matches Characters NOT in brackets
|       - Either Or
( )     - Group

Quantifiers:
*       - 0 or More
+       - 1 or More
?       - 0 or One
{3}     - Exact Number
{3,4}   - Range of Numbers (Minimum, Maximum)


#### Sample Regexs ####

[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+
"""

In [135]:
text = r"""
MetaCharacters (Need to be escaped):
. ^ $ * + ? { } [ ] \ | ( )
\d 
\D 
this is a objt.
this is a obj.
this is a objobj
this is aobj obj . 
MT5\MAM
ä90096
\123123
333333333
33333 

"""
print(text)


MetaCharacters (Need to be escaped):
. ^ $ * + ? { } [ ] \ | ( )
\d 
\D 
this is a objt.
this is a obj.
this is a objobj
this is aobj obj . 
MT5\MAM
ä90096
\123123
333333333
33333 




In [141]:
pattern = re.compile(r'333333333')
pattern

re.compile(r'333333333', re.UNICODE)

In [147]:
re.search(pattern, string = text) # return 1st match obj or None.
# re.match(pattern, string = text)  # search at the beginning of string
# re.split(pattern, string = text)

<re.Match object; span=(165, 170), match='33333'>

In [146]:
pattern = re.compile(r'33333')
re.findall(pattern, string = text)
matches = re.finditer(pattern, text)
matches
for match in matches:
    print(match)

<re.Match object; span=(165, 170), match='33333'>
<re.Match object; span=(175, 180), match='33333'>


In [149]:
pattern = re.compile(r'3{7,8}')
matches = re.finditer(pattern, text)
for match in matches:
    print(match)

<re.Match object; span=(165, 173), match='33333333'>


In [150]:
pattern = re.compile(r'\w+')
matches = re.finditer(pattern, text)
for match in matches:
    print(match)

<re.Match object; span=(1, 15), match='MetaCharacters'>
<re.Match object; span=(17, 21), match='Need'>
<re.Match object; span=(22, 24), match='to'>
<re.Match object; span=(25, 27), match='be'>
<re.Match object; span=(28, 35), match='escaped'>
<re.Match object; span=(67, 68), match='d'>
<re.Match object; span=(71, 72), match='D'>
<re.Match object; span=(74, 78), match='this'>
<re.Match object; span=(79, 81), match='is'>
<re.Match object; span=(82, 83), match='a'>
<re.Match object; span=(84, 88), match='objt'>
<re.Match object; span=(90, 94), match='this'>
<re.Match object; span=(95, 97), match='is'>
<re.Match object; span=(98, 99), match='a'>
<re.Match object; span=(100, 103), match='obj'>
<re.Match object; span=(105, 109), match='this'>
<re.Match object; span=(110, 112), match='is'>
<re.Match object; span=(113, 114), match='a'>
<re.Match object; span=(115, 121), match='objobj'>
<re.Match object; span=(122, 126), match='this'>
<re.Match object; span=(127, 129), match='is'>
<re.Match obj

In [151]:
pattern = re.compile(r'.*')
matches = re.finditer(pattern, text)
for match in matches:
    print(match)

<re.Match object; span=(0, 0), match=''>
<re.Match object; span=(1, 37), match='MetaCharacters (Need to be escaped):'>
<re.Match object; span=(37, 37), match=''>
<re.Match object; span=(38, 65), match='. ^ $ * + ? { } [ ] \\ | ( )'>
<re.Match object; span=(65, 65), match=''>
<re.Match object; span=(66, 69), match='\\d '>
<re.Match object; span=(69, 69), match=''>
<re.Match object; span=(70, 73), match='\\D '>
<re.Match object; span=(73, 73), match=''>
<re.Match object; span=(74, 89), match='this is a objt.'>
<re.Match object; span=(89, 89), match=''>
<re.Match object; span=(90, 104), match='this is a obj.'>
<re.Match object; span=(104, 104), match=''>
<re.Match object; span=(105, 121), match='this is a objobj'>
<re.Match object; span=(121, 121), match=''>
<re.Match object; span=(122, 141), match='this is aobj obj . '>
<re.Match object; span=(141, 141), match=''>
<re.Match object; span=(142, 149), match='MT5\\MAM'>
<re.Match object; span=(149, 149), match=''>
<re.Match object; span=(150

In [109]:
pattern = re.compile(r'[0-9]+.*\s*')
matches = re.finditer(pattern, text)
for match in matches:
    print(match)

<re.Match object; span=(144, 150), match='5\\MAM\n'>
<re.Match object; span=(151, 157), match='90096\n'>
<re.Match object; span=(158, 165), match='123123\n'>
<re.Match object; span=(165, 175), match='333333333\n'>
<re.Match object; span=(175, 183), match='33333 \n\n'>


In [152]:
text = """
this is a objt.
this is a obj.      # match
this is a objobj
this is aobj obj .  # match
this is aobj obj\n   # match
"""

pattern = re.compile(r'\bobj\b')
matches = re.finditer(pattern, text)
for match in matches:
    print(match)



<re.Match object; span=(27, 30), match='obj'>
<re.Match object; span=(75, 78), match='obj'>
<re.Match object; span=(103, 106), match='obj'>


In [153]:
text = """
test123@gmail.com
test.123@nccu.edu.tw
test-123@nsysu-edu-tw
"""

pattern = re.compile(r'[a-zA-Z0-9_.-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+')

matches = pattern.finditer(text)

for match in matches:
    print(match)

<re.Match object; span=(1, 18), match='test123@gmail.com'>
<re.Match object; span=(19, 39), match='test.123@nccu.edu.tw'>


In [155]:
# Positive and Negative Lookahead

text = """
qu
qb
qa
"""

pattern = re.compile(r'q(?!u)')

matches = pattern.finditer(text)

for match in matches:
    print(match)


<re.Match object; span=(4, 5), match='q'>
<re.Match object; span=(7, 8), match='q'>


In [156]:
pattern = re.compile(r'q(?=u)')

matches = pattern.finditer(text)

for match in matches:
    print(match)

<re.Match object; span=(1, 2), match='q'>


In [157]:
# Positive and Negative Lookbehind

text = """
aq
qa
"""

pattern = re.compile(r'(?<!a)q')

matches = pattern.finditer(text)

for match in matches:
    print(match)

<re.Match object; span=(4, 5), match='q'>


In [115]:
pattern = re.compile(r'(?<=a)q')

matches = pattern.finditer(text)

for match in matches:
    print(match)

<re.Match object; span=(2, 3), match='q'>


In [158]:
text = """
VT_Hedge\MAM\MAM_VUM_100084_GBP
VU_Hedge\MAM\MAM_VUM_100084_GBP
"""

pattern = re.compile(r'(?<=VT_Hedge\\)MAM')

matches = pattern.finditer(text)

for match in matches:
    print(match)

<re.Match object; span=(10, 13), match='MAM'>


In [161]:
# ASCII 
text = r"""
ä90096
90096
"""
# Match ALL ASCII Printable Characters
# Match ALL ASCII Characters (including of Non-Printable Characters)
# pattern = re.compile(r'[^ -~]')
pattern = re.compile(r'[^\x00-\x7F]')
matches = pattern.finditer(text)

for match in matches:
    print(match)


# print(re.sub(pattern = r'[^\x00-\x7F]', repl = '', string = text))

<re.Match object; span=(1, 2), match='ä'>


In [125]:
df = pd.DataFrame({
    'login' : [1, 2, 3, 4, 5, 6],
    'group' : ['VU_Hedge\MAM\MAM_VUM_100084_USD', 'VT_Hedge\MAM\MAM_VUM_100084_GBP', 'Moneta_Hedge\M_MNT_GBP', 'MNT_V_95054_GBP', 'SAM_VFX_MB_USD', 'M_MON_USD']
})
df

,login,group
0,1,VU_Hedge\MAM\MAM_VUM_100084_USD
1,2,VT_Hedge\MAM\MAM_VUM_100084_GBP
2,3,Moneta_Hedge\M_MNT_GBP
3,4,MNT_V_95054_GBP
4,5,SAM_VFX_MB_USD
5,6,M_MON_USD


In [167]:
# Series.str.contains(pat, case=True, flags=0, na=None, regex=True)
df['group'].str.contains(pat = r'\\')
# # Series.str.replace(pat, repl, n=- 1, case=None, flags=0, regex=None)
df['group'].str.replace(pat = r'\\', repl = '', regex = True)
# # Series.str.extract(pat, flags=0, expand=True)
df['group'].str.extract(pat = r'([0-9]+)')   # 等價於r'(\d+)'
df['group'].str.extract(pat = r'(\d+)')  
# # Exclude Pattern
# df['group'].str.contains(pat = r'^((?!MAM).)*$')  
df['group'].str.contains(pat = r'^(?!.*MAM).*$')
df['group'].str.contains(pat = r'^(?!.*\d).*$')




0    False
1    False
2     True
3    False
4     True
5     True
Name: group, dtype: bool

In [168]:
df = pd.read_excel('symbols.xlsx')
df.head()

,symbol,clean_symbol,symbol_type
0,ADAUSD,ADAUSD,Cryptocurrency
1,AUDCAD,AUDCAD,Forex
2,AUDCAD.i,AUDCAD,Forex
3,AUDCAD.ixp,AUDCAD,Forex
4,AUDCAD.uk+,AUDCAD,Forex


In [169]:
df['feature'] = df['symbol'].str.extract(r"([^A-Za-z0-9].*)", expand = False)
df['feature'].unique()

array([nan, '.i', '.ixp', '.uk+', '+', '.uk', '-OIL+', '#', '-OIL#',
       '.m#', '.m+', '.c', '&T', '.r', '.v', '-OIL', '-C', '-Cr', '.',
       '.Inst', '.pr', '-OIL.pr', '.pro', '.fr', '-OIL.fr', '.pr+',
       '-OIL.pr+', '.sp', '.m', '.a', '.s', '.e', '.y', '.r+', '.A',
       '.crp', '-OIL.e', '-OIL.i', '-OIL.s', '.m.e', '.p', '-OIL.p',
       '-Cs', '&T.m', '-ECN', '-STD', '-VIP', '.pr.m', '+.m', '.A.m',
       '.r.m', '-OIL-ECN', '-OIL-STD', '-OIL-VIP', '-OIL.m', '-OIL.pr.m',
       '-C.', '-C.m', '-Cr.m', '.crp.m', '-Cash', '.sc', '.b', '-OIL.',
       '-C.i', '-Cb'], dtype=object)

In [170]:
# replace(pat = r'\\', repl = '', regex = True)
df['feature2'] = df['feature'].str.replace(r'.*-OIL|.*.A|.*&T', repl = '', regex = True)
df['feature2'].unique()

array([nan, '.i', '.ixp', '.uk+', '+', '.uk', '#', '.m#', '.m+', '.c', '',
       '.r', '.v', '-C', '-Cr', '.', '.Inst', '.pr', '.pro', '.fr',
       '.pr+', '.sp', '.m', '.a', '.s', '.e', '.y', '.r+', '.crp', '.m.e',
       '.p', '-Cs', '-ECN', '-STD', '-VIP', '.pr.m', '+.m', '.r.m', '-C.',
       '-C.m', '-Cr.m', '.crp.m', '-Cash', '.sc', '.b', '-C.i', '-Cb'],
      dtype=object)

In [171]:
df['clean_symbol_regex'] = df.apply(lambda row : row['symbol'].replace(str(row['feature2']), ''), axis=1)
df_nonmatch = df[df['clean_symbol_regex'] != df['clean_symbol']]

In [172]:
df_nonmatch

,symbol,clean_symbol,symbol_type,feature,feature2,clean_symbol_regex
26,AUS200,SPI200,Index,NaN,NaN,AUS200
27,AUS200.i,SPI200,Index,.i,.i,AUS200
28,AUS200.uk,SPI200,Index,.uk,.uk,AUS200
32,BTCXBN,BTCBCH,Cryptocurrency,NaN,NaN,BTCXBN
33,BTCXET,BTCETH,Cryptocurrency,NaN,NaN,BTCXET
...,...,...,...,...,...,...
2353,DAX40ft.i,GER40ft,Future,.i,.i,DAX40ft
2394,GAS-C.i,Gas,Commodities,-C.i,-C.i,GAS
2395,GAS-Cb,Gas,Commodities,-Cb,-Cb,GAS
2396,GASOIL-C.i,Gasoil,Commodities,-C.i,-C.i,GASOIL
